In [37]:
import json 
from collections import defaultdict
import pandas as pd

def redistribute_probabilities(probs, min_value=0.001):
    zero_keys = [k for k, v in probs.items() if v == 0.0]
    if not zero_keys:
        return probs  

    zero_key = zero_keys[0]
    other_keys = [k for k in probs if k != zero_key]

    redistribute_total = 1.0 - min_value

    original_total = sum(probs[k] for k in other_keys)

    new_probs = {}
    for k in other_keys:
        new_probs[k] = probs[k] / original_total * redistribute_total

    new_probs[zero_key] = min_value

    return new_probs

def weighted_random_choice(probabilities, k):
    import random
    from collections import Counter
    probs = redistribute_probabilities(probabilities)
    outcomes = list(probs.keys())
    weights = list(probs.values())
    return Counter(random.choices(outcomes, weights=weights, k=k)).most_common(1)[0][0]


def run_season_sim(teams, home_probs):

    team_dict = defaultdict(lambda:0)

    for t1 in teams:
        for t2 in teams:
            if t1 != t2:
                if t1 not in home_probs:
                    result = weighted_random_choice(home_probs["Promoted"][t2], 10)
                if t2 not in home_probs:
                    result = weighted_random_choice(home_probs[t1]["Promoted"], 10)
                else:
                    result = weighted_random_choice(home_probs[t1][t2], 10)
                    
                if result == "Win":
                    team_dict[t1] += 3
                elif result == "Loss":
                    team_dict[t2] += 3
                else:
                    team_dict[t1] += 1
                    team_dict[t2] += 1
    
    return sorted(team_dict, key=lambda k: team_dict[k], reverse=True)

def run_sim(season, k):
        
    with open(f".\\data\\{season}\\home_team_matchup_probs_rollback_2003.json", "r", encoding="utf-8") as f:
        probs = json.load(f)

        
        
    teams = pd.read_csv(f".\\data\\{season}\\{season}_table.csv", encoding="utf-8")["Team"].to_list()
    
    sims = []
    
    for _ in range(k):
        sims.append(run_season_sim(teams, probs))
        
    team_dict = defaultdict(lambda:0)
    
    for sim in sims:
        team_dict[sim[0]] +=1       
        
    return sorted(team_dict, key=lambda k: team_dict[k], reverse=True)

season = 2023

run_sim(season, 100)

['Flamengo',
 'Palmeiras',
 'Internacional',
 'Sao Paulo',
 'Corinthians',
 'Cruzeiro',
 'Gremio',
 'Atletico-MG',
 'Fluminense']